In [1]:
import pyotp
import configparser
from smartapi import SmartConnect
import pandas as pd
from datetime import datetime
import requests
import numpy as np
import math
from datetime import *
import re
from time import sleep
from time import time as tim
from bs4 import BeautifulSoup
from IPython.display import clear_output
import nsepython
import concurrent
import inflect
from time import strptime


config = configparser.ConfigParser()

# # nishad  cred
# token = 'S3N5FR3JZ6OSTM3PBPHJRNWEP4'
# config.read('my_credentials.ini')

# shahabaz cred
token = 'OTXOCWZA4CU4YP7LUUZCZLOCYM'
config.read('shahabas_credentials.ini')

client_id = config['angel']['client_id']
password = config['angel']['password']
api_key = config['angel']['api_key']

def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = 'test inside fun'
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})

# fetch token/instrument from dataframe
def getTokenInfo (symbol,exch_seg='NSE',instrumenttype='OPTIDX',strike_price='',pe_ce='CE',expiry=None):
    df = token_df
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])

intializeSymbolTokenMap()
obj=SmartConnect(api_key=api_key)
data = obj.generateSession(client_id,password,pyotp.TOTP(token).now())
refreshToken= data['data']['refreshToken']
feedToken=obj.getfeedToken()
userProfile= obj.getProfile(refreshToken)
print(userProfile)

{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'M368278', 'name': 'MOIDEEN KM', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


In [2]:
symbol = 'BANKNIFTY'
payload = nsepython.nse_optionchain_scrapper(symbol)
currentExpiry,dte = nsepython.nse_expirydetails(payload,0)
p = inflect.engine()
day = currentExpiry.strftime("%d")
month = currentExpiry.strftime("%B")
day = p.ordinal(day)
expiry_folder_name = 'Expiry ' + day +' '+ month + '.csv'
expiry_folder_name

'Expiry 15th December.csv'

In [6]:
def fetch_single_strike_history_and_save_to_csv(symbol,strike):
    q = []
    for i in range(22, 0, -1):
        f = currentExpiry - timedelta(days=i)
        t = currentExpiry - timedelta(days=i-1)
        historicParam = {
             "exchange": "NFO",
             "symboltoken": symbol,
             "interval": "ONE_MINUTE",
             "fromdate": str(f) + " 09:15",
             "todate": str(t) + " 15:30"
        }
        try:
            x = obj.getCandleData(historicParam)
            sleep(0.1)
        except:
            try:
                x = obj.getCandleData(historicParam)
                sleep(1)
            except:
                continue
        if x['data'] != None:
            q += x['data']
    df = pd.DataFrame(q)
    if not df.empty:
        df['date'] = df[0].apply(lambda x: x[:10])
        df[0] = df[0].apply(lambda x: x[11:16])
        first_column = df.pop('date')
        df.insert(0, 'date', first_column)
        df['date'] = df['date'].apply(lambda x: x.replace('-','/'))
        ind = 'BANKNIFTY' + strike
        df['ind'] = ind
        df.set_index('ind',inplace=True)
        df.columns = df.iloc[0]
        df.to_csv(f'Expiry 15th December/{ind}.csv',header=False)
        x = pd.read_csv(f'Expiry 15th December/{ind}.csv')
        print(ind,x.index.has_duplicates)

    


In [8]:
for i in range(50000,70000,100):
    ce = getTokenInfo(symbol,'NFO','OPTIDX',i,'CE')
    pe = getTokenInfo(symbol,'NFO','OPTIDX',i,'PE')
    ce = ce[ce['expiry'] == str(currentExpiry)]
    pe = pe[pe['expiry'] == str(currentExpiry)]
    if not ce.empty:
        strike = str(i) + 'CE'
        ce_token = ce.iloc[0]['token']
        fetch_single_strike_history_and_save_to_csv(ce_token,strike)
    if not pe.empty:
        strike = str(i) + 'PE'
        pe_token = pe.iloc[0]['token']
        fetch_single_strike_history_and_save_to_csv(pe_token,strike)